# Genetic algorithm skeleton

I made a small skeleton for genetic algorithms, a while back. I've decided to put it online because it might help some people.  The goal is to find a solution with the highest fitness function (the function we try to maximize). Above each cell I will explain the function of the object within the genetic algorithm concept and how to use it.

**WARNING:** This is not very well tested, but it seems to work.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
from collections import defaultdict
from random import shuffle
from sklearn import manifold
from sklearn import decomposition
from copy import copy
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd

## Individual

Individuals are proposed solutions that can breed with other individuals, mutate into new individuals and their strength is measured by the fitness function (we try to find the optimal individual with the highest fitness). Initializer is a function that generates a new, random individual. The fitness function evaluates the fitness of this individual. The chromosome parameterizes the solution and will be used for changing solutions and for breeding.

In [ ]:
class Individual:
    def __init__(self, initializer, fitness_function, chromosome=None, **kwargs):
        self.initializer = initializer
        self.fitness_function = fitness_function
        self.fitness_score = None
        if chromosome is not None:
            self.chromosome = chromosome
        else:
            self.chromosome = initializer.initialize(**kwargs)
        self.fitness_score = self.fitness_function.fitness(self.chromosome)
    
    def get_chromosome(self):
        if self.chromosome is not None:
            return self.chromosome
        else:
            raise ValueError('Chromosome not initialized')
    
    def set_chromosome(self, chromosome):
        self.chromosome = chromosome
        
    def mutate(self, mutation_operator, **kwargs):
        self.chromosome = mutation_operator.mutate(self.chromosome, **kwargs)
        self.fitness_score = self.fitness_function.fitness(self.chromosome)
    
    def set_fitness(self):
        self.fitness_score = self.fitness_function.fitness(self.chromosome)
        return self.fitness_score
        
    def get_fitness(self):
        if self.fitness_score is not None:
            return self.fitness_score
        else:
            raise ValueError('Fitness score is not set yet')
            
    def deep_copy(self):
        return Individual(self.initializer, self.fitness_function, chromosome = self.chromosome)

## Generation

A generation is a group of individuals. They breed, evolve/mutate and have a fight for survival where the fittest will survive. Crossover determines how to combine two (or more, although this implementation does not allow for this) individuals. Selection determines what parents to use for breeding. We select the same amount of parents as the population size, but certain individuals will be parents multiple times while other will not be. Selection determines how to select them, and is usually based on just the fitness function but it can also take other factors in consideration, like variety.

In [ ]:
class Generation:
    def __init__(self, fitness_function, initializer, selection, crossover, mutation,
                 members=None, number_members=0, maximize=True, **kwargs):    
        self.fitness_function = fitness_function
        self.initializer = initializer
        self.crossover = crossover
        self.mutation = mutation
        self.selection = selection
        self.maximize = maximize
        self.members = []
        
        if members is None:
            self.number_members = number_members
            for i in range(number_members):
                self.members.append(Individual(self.initializer, fitness_function, **kwargs))
        else:
            self.number_members = len(members)
            self.members = members
            
        self.evaluate_members()
            
    def evaluate_members(self):
        for member in self.members:
            member.set_fitness()
        self.members = sorted(self.members, key=lambda x: -x.get_fitness())
    
    def breed(self, **kwargs):
        parent_selection = self.selection.select(self.members, **kwargs)
        children = []
        for parents in zip(parent_selection[::2],parent_selection[1::2]):
            child_1, child_2 = self.crossover.cross(parents[0], parents[1], **kwargs)
            child_1.mutate(self.mutation, **kwargs)
            child_2.mutate(self.mutation, **kwargs)
            children.append(child_1)
            children.append(child_2)
        return Generation(self.fitness_function, self.initializer, self.selection,
                          self.crossover, self.mutation, members=children, **kwargs)
    
    def get_top_score(self):
        return self.members[0].get_chromosome(), self.members[0].get_fitness()
    
    def get_top_member(self):
        return self.members[0]

## Genetic optimization

This is the optimizer itself, with all the hyperparameters and genetic operators involved. The ```.optimize``` starts finding solutions while there are a few plot functions available that can show progress over time and can project the generations over time onto a 2 dimensional animation.

In [ ]:
class GeneticOptimization:
    def __init__(self, fitness_function, individuals_in_generation, number_generations, 
                 initializer=10, selection='roulette', crossover='uniform', mutation='random', members=None, number_members=0, 
                 stop_fitness=None, **kwargs):
        self.kwargs = kwargs
        self.initializer = get_initializer(initializer, **kwargs)
        self.crossover = get_crossover(crossover, **kwargs)
        self.mutation = get_mutation(mutation, **kwargs)
        self.selection = get_selection(selection, **kwargs)
        self.fitness_function = fitness_function
        self.individuals_in_generation = individuals_in_generation
        self.number_generations = number_generations
        self.top_fitness = -10**(300)
        self.top_member = None
        self.generations = []
        self.projection_model = None
        self.stop_fitness = stop_fitness
        
    def optimize(self, verbose = True, keep_generations=False):
        current_generation = Generation(self.fitness_function, self.initializer, self.selection, self.crossover, 
                                        self.mutation, number_members = self.individuals_in_generation, **self.kwargs)
        if keep_generations:
            self.generations.append(current_generation)
        fitness_values = [current_generation.get_top_member().get_fitness()]
        top_member = current_generation.get_top_member()
        top_fitness = top_member.get_fitness()
        if top_fitness>self.top_fitness:
            self.top_fitness = top_member.get_fitness()
            self.top_member = top_member
        for i in range(self.number_generations-1):
            next_generation = current_generation.breed(**self.kwargs)
            if keep_generations:
                self.generations.append(next_generation)
            top_member = next_generation.get_top_member()
            top_fitness = top_member.get_fitness()
            fitness_values.append(top_fitness)
            if top_fitness>self.top_fitness:
                self.top_fitness = top_member.get_fitness()
                self.top_member = top_member
            current_generation = next_generation
            if verbose:
                print('Generation '+str(i+1), top_fitness, self.top_fitness)
        print('Best solution:')
        print(self.top_member.get_chromosome())
        print(self.top_fitness)
        self.fitness_values = fitness_values
        return self.top_member, self.top_fitness, fitness_values
    
    def plot_fitness(self, max_fitness=False, title=None):
        x = range(1,len(self.fitness_values)+1)
        fitness_values = self.fitness_values
        max_fitness_values = []
        max_value = -(10**100)
        for fitness_value in self.fitness_values:
            if fitness_value>max_value:
                max_value = fitness_value
            max_fitness_values.append(max_value)
        plt.plot(x,fitness_values)
        plt.plot(x,max_fitness_values)
        plt.show()
        
    def plot_projection_animation(self, proj_chroms, members_per_gen, optimal_chrom):
        proj_x = map(lambda x:x[0],proj_chroms)
        proj_y = map(lambda x:x[1],proj_chroms)
        x_min, x_max = min(proj_x), max(proj_x)
        y_min, y_max = min(proj_y), max(proj_y)
        x_range, y_range = x_max-x_min, y_max-y_min 
        
        fig = plt.figure(figsize=(7, 7))
        ax = fig.add_axes([0, 0, 1, 1], frameon=False)
        ax.set_xlim(x_min-x_range/15., x_max+x_range/15.), ax.set_xticks([])
        ax.set_ylim(y_min-y_range/15., y_max+y_range/15.), ax.set_yticks([])
        
        if optimal_chrom is not None:
            chroms_pos = np.zeros(members_per_gen+1, dtype=[('position', float, 2)])
            chroms_pos['position'][:members_per_gen] = zip(proj_x[:members_per_gen],proj_y[:members_per_gen])
            chroms_pos['position'][members_per_gen] = proj_chroms[-1]
        else:
            chroms_pos = np.zeros(members_per_gen, dtype=[('position', float, 2)])
            chroms_pos['position'][:members_per_gen] = zip(proj_x[:members_per_gen],proj_y[:members_per_gen])
        
        scat = ax.scatter(chroms_pos['position'][:, 0], chroms_pos['position'][:, 1], lw=0.5,)
    
        def update(frame_number):
            chroms_pos['position'][:members_per_gen] = zip(proj_x[members_per_gen*frame_number:(frame_number+1)*members_per_gen],
                                                           proj_y[members_per_gen*frame_number:(frame_number+1)*members_per_gen])
            scat.set_offsets(chroms_pos['position'])
            
        ani = animation.FuncAnimation(fig, update, frames=len(self.generations), interval=500, repeat=False)
        plt.show()
        
    def plot_generations(self, members_per_gen=None, optimal_chrom=None, proj_method='t-sne'):
        all_members = []
        for generation in self.generations:
            if members_per_gen is None:
                all_members += generation.members
            else:
                cur_members = copy(generation.members)
                shuffle(cur_members)
                all_members += cur_members[:members_per_gen]
        all_chromosomes = list(map(lambda x: x.get_chromosome(), all_members))
        if members_per_gen is None:
            members_per_gen = len(generation.members)
        if optimal_chrom is not None:
            all_chromosomes += [optimal_chrom]
        if proj_method=='t-sne':
            tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
            proj_chroms = tsne.fit_transform(all_chromosomes)
        elif proj_method=='pca':
            pca = decomposition.PCA(n_components=2)
            proj_chroms = pca.fit_transform(all_chromosomes)
        else:
            raise ValueError('Unrecognized projection method')
        self.plot_projection_animation(proj_chroms, members_per_gen, optimal_chrom)

## Object templates

Here are the object templates that you need to implement to implement this for your own optimization problem.

- Initializer: Object that generates random chromosomes that you use to make new individuals via ```initialize```
- Crossover: Object that has a ```cross``` method to cross two parents and generate two new individuals
- RouletteSelection: Object that takes a list of members in ```select``` and generates a new list of members according to their fitness
- Mutation: Object that uses ```mutate``` to (potentially) mutate an individual to create diversity and allow to search in new regions of the solution space
- Fitness: Object that has the method ```fitness``` which evaluates the fitnes function of this individual. If you need caching you can build it in here

In [ ]:
class Initializer:
    def __init__(self):
        pass
    
    def initialize():
        pass
    
class BitInitializer(Initializer):
    def __init__(self, number_genes, **kwargs):
        self.number_genes = number_genes
        
    def initialize(self, **kwargs):
        return tuple(np.random.choice([0,1],self.number_genes))
    
def get_initializer(initializer, **kwargs):
    if isinstance(initializer, int):
        return BitInitializer(initializer)
    elif isinstance(initializer, Initializer):
        return initializer
    else:
        raise ValueError('No proper initializer set')

class Crossover:
    def __init__(self):
        pass
    
    def cross():
        pass
    
class UniformCrossover(Crossover):
    def __init__(self, crossover_ratio=None, **kwargs):
        if crossover_ratio is None:
            self.prob_dist = None
        else:
            if isinstance(ratio, float):
                self.prob_dist = [ratio, 1-ratio]
            else:
                raise ValueError('Random crossover ratio should be None, a float or "fitness_ratio"')
    
    def cross(self, parent_1, parent_2, **kwargs):
        zipped_parents = list(zip(parent_1.get_chromosome(), parent_2.get_chromosome(),
                                  np.random.choice([0,1], len(parent_1.get_chromosome()), p=self.prob_dist)))
        chromosome_1 = [x[x[2]] for x in zipped_parents]
        chromosome_2 = [x[abs(1-x[2])] for x in zipped_parents]
        #print(chromosome_1)
        #print(chromosome_2)
        return (Individual(parent_1.initializer, parent_1.fitness_function, chromosome_1, **kwargs), 
                Individual(parent_1.initializer, parent_1.fitness_function, chromosome_2, **kwargs))
            
def get_crossover(crossover, **kwargs):
    if isinstance(crossover, str):
        if crossover=='uniform':
            return UniformCrossover(**kwargs)
        else:
            raise ValueError('Crossover method '+crossover+' is unknown')
    elif isinstance(crossover,Crossover):
        return crossover
    else:
        raise ValueError("No proper crossover selected")
        
class Selection:
    def __init__(self):
        pass
    
    def select(self, members):
        pass
    
class RouletteSelection(Selection):
    def __init__(self, min_prob_fraction=0., **kwargs):
        self.min_prob_fraction = min_prob_fraction
        
    def select(self, members, **kwargs):
        lowest_fitness = min(map(lambda x: x.get_fitness(),members))
        total_shifted_fitness = sum(map(lambda x: x.get_fitness()-lowest_fitness,members))
        norm_cum_fitness = []
        total_prob = 0.
        total_fitness = 0.
        for member in members:
            total_prob += (1-self.min_prob_fraction)*(member.get_fitness()-lowest_fitness)/float(total_shifted_fitness)+self.min_prob_fraction/float(len(members))
            norm_cum_fitness.append((member,total_prob))
            total_fitness += member.get_fitness()
        next_generation_selection = []
        new_total_fitness = 0.
        for i in range(len(members)):
            p = np.random.random_sample()
            for candidate in norm_cum_fitness:
                if p<candidate[1]:
                    next_generation_selection.append(candidate[0].deep_copy())
                    new_total_fitness += next_generation_selection[-1].get_fitness()
                    break
        #print total_fitness
        #print new_total_fitness
        #print len(next_generation_selection)
        shuffle(next_generation_selection)
        return next_generation_selection
    
class TruncationSelection(Selection):
    def __init__(self, top_fraction=None, top_number=None, **kwargs):
        self.top_fraction = top_fraction
        self.top_number = top_number
        
    def select(self, members, **kwargs):
        if self.top_fraction is not None:
            top_number = int(round(len(members)*self.top_fraction))
        else:
            top_number = self.top_number
        #print top_number
        #total_fitness = 0.
        #for member in members:
        #    total_fitness += member.get_fitness()
        parents_pool = members[:top_number]
        next_total_fitness = 0.
        next_generation_selection = [x.deep_copy() for x in np.random.choice(parents_pool, len(members))]
        #for member in next_generation_selection:
        #    next_total_fitness += member.get_fitness()
        #print total_fitness
        #print next_total_fitness
        return next_generation_selection

def get_selection(selection, **kwargs):
    if isinstance(selection, str):
        if selection=='roulette':
            return RouletteSelection(**kwargs)
        elif selection=='truncate':
            return TruncationSelection(**kwargs)
        else:
            raise ValueError('Selection method '+selection+' is unknown')
    elif isinstance(selection,Selection):
        return selection
    else:
        raise ValueError("No proper selection selected")
        
class Mutation:
    def __init__(self):
        pass
    
    def mutate(self):
        pass
    
class RandomMutation(Mutation):
    def __init__(self, mutation_fraction=0.01, **kwargs):
        self.mutation_fraction = mutation_fraction
        
    def mutate(self, chromosome, **kwargs):
        return tuple(gene if np.random.random_sample()>self.mutation_fraction else abs(1-gene) for gene in chromosome)

def get_mutation(mutation, **kwargs):
    if isinstance(mutation, str):
        if mutation=='random':
            return RandomMutation(**kwargs)
        else:
            raise ValueError('Mutation method '+mutation+' is unknown')
    elif isinstance(mutation,Mutation):
        return mutation
    else:
        raise ValueError("No proper mutation selected")

class Fitness:
    def __init__(self, fitness_function, **kwargs):
        self.fitness_function = fitness_function
    
    def fitness(self, chromosome, **kwargs):
        return self.fitness_function(chromosome)

## Quick, easy example using implementations above

We have a 100 items with different weights, for every item we can choose to take it with us or not, and we are looking for a configuration where we take half of the weight with us, or as close as possible. Our fitness function calculates how far we are off from half and negates it (we want to maximize fitness), initializer generates random bit strings of length 100, we use roulette selection (sample parents weighted according to their fitness) and the ```mutation_fraction = 0.3``` means we flip 30% of the bits at random (very high)

In [ ]:
weights = list(range(1,34)) + list(range(int(round(1.5*(33**2)+0.5*33)),int(round(1.5*(33**2)+2.5*33)+1)))
weights_sum = sum(weights)

def fitness_fun(chromosome):
    return -abs(weights_sum/2.0-np.dot(chromosome,weights))

optimization = GeneticOptimization(Fitness(fitness_fun), 500, 100, initializer=100, selection='roulette', mutation_fraction=0.3, min_prob_fraction=0.)
solution, fitness_value, fitness_values = optimization.optimize(verbose=True, keep_generations=True)
optimization.plot_fitness()

In [ ]:
optimization.plot_fitness()